<a href="https://colab.research.google.com/github/laribar/bitcoinprediction/blob/main/modelo_yfinance_geral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import logging
from scipy.stats import norm

# Configuração de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Função para baixar dados e calcular métricas técnicas
def baixar_dados(acao, intervalo="1d", periodo="2y"):
    try:
        # Baixar dados da ação
        df = yf.download(acao, period=periodo, interval=intervalo, progress=False, auto_adjust=True)
        if df.empty or df.isnull().values.any():
            logging.error(f"Erro: Dados faltantes ou inválidos para {acao}.")
            return None

        preco_coluna = "Close"

        if preco_coluna not in df.columns:
            logging.error(f"Erro ao processar {acao}: Nenhuma coluna de fechamento encontrada nos dados.")
            return None

        # Remove NaNs
        df.dropna(inplace=True)

        # Médias Móveis
        df["SMA_9"] = df[preco_coluna].rolling(window=9).mean().shift(1)
        df["SMA_21"] = df[preco_coluna].rolling(window=21).mean().shift(1)
        df["EMA_50"] = df[preco_coluna].ewm(span=50, adjust=False).mean().shift(1)

        # Bandas de Bollinger
        df["STD"] = df[preco_coluna].rolling(window=20).std().shift(1)
        df["Upper_Band"] = df["SMA_21"] + (df["STD"] * 2)
        df["Lower_Band"] = df["SMA_21"] - (df["STD"] * 2)

        # RSI (Índice de Força Relativa)
        delta = df[preco_coluna].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df["RSI"] = 100 - (100 / (1 + rs)).shift(1)

        # MACD (Moving Average Convergence Divergence)
        df["EMA_12"] = df[preco_coluna].ewm(span=12, adjust=False).mean()
        df["EMA_26"] = df[preco_coluna].ewm(span=26, adjust=False).mean()
        df["MACD"] = df["EMA_12"] - df["EMA_26"]
        df["MACD_Signal"] = df["MACD"].ewm(span=9, adjust=False).mean()
        df["MACD_Histogram"] = df["MACD"] - df["MACD_Signal"]

        # ADX (Average Directional Index)
        df["TR"] = np.maximum(
            df["High"] - df["Low"],
            np.maximum(abs(df["High"] - df[preco_coluna].shift(1)), abs(df["Low"] - df[preco_coluna].shift(1)))
        )
        df["ATR"] = df["TR"].rolling(window=14).mean()
        df["Plus_DM"] = np.where(
            (df["High"] - df["High"].shift(1)) > (df["Low"].shift(1) - df["Low"]),
            np.maximum(df["High"] - df["High"].shift(1), 0),
            0
        )
        df["Minus_DM"] = np.where(
            (df["Low"].shift(1) - df["Low"]) > (df["High"] - df["High"].shift(1)),
            np.maximum(df["Low"].shift(1) - df["Low"], 0),
            0
        )
        df["Plus_DI"] = (df["Plus_DM"].rolling(window=14).mean() / df["ATR"]) * 100
        df["Minus_DI"] = (df["Minus_DM"].rolling(window=14).mean() / df["ATR"]) * 100
        df["DX"] = (abs(df["Plus_DI"] - df["Minus_DI"]) / (df["Plus_DI"] + df["Minus_DI"])) * 100
        df["ADX"] = df["DX"].rolling(window=14).mean().shift(1)

        # Estocástico (%K e %D)
        df["%K"] = ((df[preco_coluna] - df["Low"].rolling(window=14).min()) /
                   (df["High"].rolling(window=14).max() - df["Low"].rolling(window=14).min())) * 100
        df["%D"] = df["%K"].rolling(window=3).mean().shift(1)

        # ATR (Average True Range)
        df["ATR"] = df["TR"].rolling(window=14).mean().shift(1)

        # VIX (Índice de Volatilidade)
        vix = yf.download("^VIX", period=periodo, interval=intervalo, progress=False)
        df["VIX"] = vix["Close"].shift(1)

        # Remove colunas auxiliares que não serão usadas no modelo
        df = df.drop(columns=["STD", "TR", "Plus_DM", "Minus_DM", "DX", "EMA_12", "EMA_26"])

        return df

    except Exception as e:
        logging.error(f"Erro ao baixar dados para {acao}: {e}", exc_info=True)
        return None

# Função para calcular a probabilidade de atingir o preço alvo
def calcular_probabilidade(preco_atual, preco_alvo, volatilidade, dias):
    """
    Calcula a probabilidade de atingir o preço alvo com base na volatilidade e no horizonte de tempo.
    """
    # Garantir que preco_atual e volatilidade sejam valores escalares
    if isinstance(preco_atual, pd.Series):
        preco_atual = preco_atual.item()  # Converte para valor escalar
    if isinstance(volatilidade, pd.Series):
        volatilidade = volatilidade.item()  # Converte para valor escalar

    if preco_atual == 0 or volatilidade == 0:
        return 0.0

    T = dias / 252  # Converter dias para anos (252 dias úteis no ano)
    d1 = (np.log(preco_alvo / preco_atual)) / (volatilidade * np.sqrt(T))
    probabilidade = norm.cdf(d1)
    return probabilidade

# Função para treinar o modelo e prever preços futuros com probabilidades
def prever_precos_com_probabilidade(df, acao, intervalo):
    try:
        # Definir target como o preço do dia seguinte
        df["Target"] = df["Close"].shift(-1)
        df.dropna(inplace=True)  # Remove a última linha com target NaN

        # Selecionar features e target
        features = [
            'SMA_9', 'SMA_21', 'EMA_50', 'Upper_Band', 'Lower_Band', 'RSI',
            'MACD', 'MACD_Signal', 'MACD_Histogram', 'ADX', '%K', '%D', 'ATR', 'VIX'
        ]
        X = df[features]
        y = df['Target'].values.ravel()  # Converter y para array 1D

        # Separar os últimos 10% dos dados para teste
        test_size = int(0.1 * len(df))
        X_train, X_test = X.iloc[:-test_size], X.iloc[-test_size:]
        y_train, y_test = y[:-test_size], y[-test_size:]

        # Verificar se há dados suficientes para treinamento e teste
        if len(X_train) == 0 or len(X_test) == 0:
            logging.error(f"Erro: Dados insuficientes para treinamento/teste em {acao} no intervalo {intervalo}")
            return None, None

        # Padronizar os dados
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Ajuste de hiperparâmetros com GridSearchCV e TimeSeriesSplit
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        }
        model = RandomForestRegressor(random_state=42)
        tss = TimeSeriesSplit(n_splits=5)  # Usar 5 splits para validação cruzada
        grid_search = GridSearchCV(model, param_grid, cv=tss, scoring='neg_mean_squared_error')
        grid_search.fit(X_train_scaled, y_train)

        best_model = grid_search.best_estimator_

        # Avaliar o modelo no conjunto de teste
        y_pred = best_model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        logging.info(f"Desempenho do modelo para {acao}: MSE = {mse:.2f}, MAE = {mae:.2f}")

        # Criar datas futuras
        future_dates = [df.index[-1] + pd.Timedelta(days=i) for i in range(1, 6)]

        # Criar features futuras com base em tendências
        future_features = pd.DataFrame({
            'SMA_9': [df['SMA_9'].iloc[-1]] * 5,
            'SMA_21': [df['SMA_21'].iloc[-1]] * 5,
            'EMA_50': [df['EMA_50'].iloc[-1]] * 5,
            'Upper_Band': [df['Upper_Band'].iloc[-1]] * 5,
            'Lower_Band': [df['Lower_Band'].iloc[-1]] * 5,
            'RSI': [df['RSI'].iloc[-1]] * 5,
            'MACD': [df['MACD'].iloc[-1]] * 5,
            'MACD_Signal': [df['MACD_Signal'].iloc[-1]] * 5,
            'MACD_Histogram': [df['MACD_Histogram'].iloc[-1]] * 5,
            'ADX': [df['ADX'].iloc[-1]] * 5,
            '%K': [df['%K'].iloc[-1]] * 5,
            '%D': [df['%D'].iloc[-1]] * 5,
            'ATR': [df['ATR'].iloc[-1]] * 5,
            'VIX': [df['VIX'].iloc[-1]] * 5
        }, index=future_dates)

        # Transformar features futuras usando o scaler ajustado
        future_features_scaled = scaler.transform(future_features)

        # Fazer previsões
        future_predictions = best_model.predict(future_features_scaled)

        # Calcular volatilidade histórica (desvio padrão dos retornos diários)
        retornos = df['Close'].pct_change().dropna()
        volatilidade = retornos.std() * np.sqrt(252)  # Volatilidade anualizada

        # Calcular probabilidades para cada previsão
        probabilidades = []
        for i, previsao in enumerate(future_predictions):
            dias = i + 1  # Horizonte de tempo em dias
            probabilidade = calcular_probabilidade(df['Close'].iloc[-1], previsao, volatilidade, dias)
            probabilidades.append((previsao, probabilidade, dias))

        return future_predictions, probabilidades

    except Exception as e:
        logging.error(f"Erro ao prever preços para {acao}: {e}", exc_info=True)
        return None, None

# Função para gerar alertas de compra/venda com probabilidades
def gerar_alertas(df, previsoes, probabilidades, acao, intervalo):
    try:
        # Garantir que last_price seja um valor escalar
        last_price = df['Close'].iloc[-1]
        if isinstance(last_price, pd.Series):
            last_price = last_price.item()  # Converte para um valor escalar

        if previsoes is not None and len(previsoes) > 0:
            avg_future_price = np.mean(previsoes)

            # Gerar alerta e justificativa
            if avg_future_price > last_price * 1.02:  # Tendência de alta
                alerta = '🚀 Compra sugerida'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) é 2% maior que o preço atual ({last_price:.2f}). "
                    f"Isso indica uma tendência de alta."
                )
            elif avg_future_price < last_price * 0.98:  # Tendência de baixa
                alerta = '⚠️ Venda sugerida'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) é 2% menor que o preço atual ({last_price:.2f}). "
                    f"Isso indica uma tendência de baixa."
                )
            else:
                alerta = '🔍 Acompanhar'
                justificativa = (
                    f"A média das previsões futuras ({avg_future_price:.2f}) está dentro de 2% do preço atual ({last_price:.2f}). "
                    f"Isso sugere que o mercado está estável no momento."
                )
        else:
            alerta = '❌ Sem previsões disponíveis'
            justificativa = 'Não há previsões disponíveis para esta ação.'

        # Exibir resultados
        print(f"Stock: {acao} | Intervalo: {intervalo}")
        print(f"Last price: {last_price:.2f}")
        print(f"Predictions: {previsoes}")
        print(f"Alert: {alerta}")
        print(f"Justificativa: {justificativa}")

        # Exibir probabilidades
        if probabilidades is not None:
            print("\nProbabilidades de Atingir o Preço Alvo:")
            for previsao, probabilidade, dias in probabilidades:
                print(f"Previsão: {previsao:.2f} | Probabilidade: {probabilidade * 100:.2f}% | Dias: {dias}")

        print("-" * 80)

    except Exception as e:
        logging.error(f"Erro ao gerar alertas para {acao}: {e}", exc_info=True)

# Função para obter o Top 10 da NASDAQ por capitalização de mercado
def obter_top_10_nasdaq():
    """
    Obtém o Top 10 da NASDAQ por capitalização de mercado.
    """
    # Baixar a lista de tickers da NASDAQ
    nasdaq_tickers = pd.read_html("https://en.wikipedia.org/wiki/Nasdaq-100")[4]
    top_10 = nasdaq_tickers.head(3)['Ticker'].tolist()
    return top_10

# Processar ações
acoes = obter_top_10_nasdaq()
for acao in acoes:
    df = baixar_dados(acao)
    if df is not None:
        previsoes, probabilidades = prever_precos_com_probabilidade(df, acao, "1d")
        gerar_alertas(df, previsoes, probabilidades, acao, "1d")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: ADBE | Intervalo: 1d
Last price: 389.61
Predictions: [434.79028123 434.79028123 434.79028123 434.79028123 434.79028123]
Alert: 🚀 Compra sugerida
Justificativa: A média das previsões futuras (434.79) é 2% maior que o preço atual (389.61). Isso indica uma tendência de alta.

Probabilidades de Atingir o Preço Alvo:
Previsão: 434.79 | Probabilidade: 100.00% | Dias: 1
Previsão: 434.79 | Probabilidade: 99.98% | Dias: 2
Previsão: 434.79 | Probabilidade: 99.78% | Dias: 3
Previsão: 434.79 | Probabilidade: 99.32% | Dias: 4
Previsão: 434.79 | Probabilidade: 98.64% | Dias: 5
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


Stock: AMD | Intervalo: 1d
Last price: 107.14
Predictions: [108.47992994 108.47992994 108.47992994 108.47992994 108.47992994]
Alert: 🔍 Acompanhar
Justificativa: A média das previsões futuras (108.48) está dentro de 2% do preço atual (107.14). Isso sugere que o mercado está estável no momento.

Probabilidades de Atingir o Preço Alvo:
Previsão: 108.48 | Probabilidade: 66.30% | Dias: 1
Previsão: 108.48 | Probabilidade: 61.69% | Dias: 2
Previsão: 108.48 | Probabilidade: 59.59% | Dias: 3
Previsão: 108.48 | Probabilidade: 58.33% | Dias: 4
Previsão: 108.48 | Probabilidade: 57.46% | Dias: 5
--------------------------------------------------------------------------------
Stock: ABNB | Intervalo: 1d
Last price: 126.15
Predictions: [120.68930519 120.68930519 120.68930519 120.68930519 120.68930519]
Alert: ⚠️ Venda sugerida
Justificativa: A média das previsões futuras (120.69) é 2% menor que o preço atual (126.15). Isso indica uma tendência de baixa.

Probabilidades de Atingir o Preço Alvo:
Previsã

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
